# Face tracking pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and tracking on an image dataset using MTCNN.

In [ ]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display
from matplotlib import image
from util import Mosaic, DrawRectImg
from ml_part import Detection, Recognition
from detection import mtcnn_detection, mtcnn_get_embeddings, mtcnn_recognition

In [ ]:

def ProcessImage(img, bbox_thr, recog_thr, device):
    # Object Detection
    # bboxes = Detection(img, bbox_thr)
    bboxes, landmarks = mtcnn_detection(img, bbox_thr, device)
    # print(bboxes)
    faces, unknown_embeddings = mtcnn_get_embeddings(img, bboxes, device)

    # Object Recognition
    # unknown_bboxes, face_ids = Recognition(img, bboxes, recog_thr)
    face_ids, result_probs = mtcnn_recognition(img, unknown_embeddings, recog_thr, device)
    # print(face_ids)

    # Mosaic
    # img = Mosaic(img, bboxes, face_ids, n=3, mode=0) 

    # 특정인에 bbox와 name을 보여주고 싶으면
    processed_img = DrawRectImg(img, bboxes, landmarks, face_ids)

    return processed_img
    # return img

#### Determine if an nvidia GPU is available

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

#### Define MTCNN module

Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [ ]:
mtcnn = MTCNN(keep_all=True, device=device)

#### Get a sample video

We begin by loading a video with some faces in it. The `mmcv` PyPI package by mmlabs is used to read the video frames (it can be installed with `pip install mmcv`). Frames are then converted to PIL images.

In [ ]:
video = mmcv.VideoReader('video.mp4')
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

display.Video('video.mp4', width=640)

#### Run video through MTCNN

We iterate through each frame, detect faces, and draw their bounding boxes on the video frames.

In [ ]:
frames_tracked = []
for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')
    
    frame_draw = ProcessImage(frame, 20, 0.8, device)
    # Add to frame list
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\nDone')

#### Display detections

In [10]:
d = display.display(frames_tracked[0], display_id=True)
i = 1
try:
    while True:
        d.update(frames_tracked[i % len(frames_tracked)])
        i += 1
except KeyboardInterrupt:
    pass

#### Save tracked video

In [ ]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('video_tracked.mp4', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()